<a href="https://colab.research.google.com/github/savindu29/Save-hearts/blob/feat%2Fsavindu/lstm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dataset = "/content/drive/MyDrive/data/data.csv"


In [32]:

import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator


In [35]:
filename = "/content/drive/MyDrive/data/data.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    60 non-null     object
 1   Amount  60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None


In [37]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)


In [40]:
amount_data = df['Amount'].values
amount_data = amount_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(amount_data))

amount_train = amount_data[:split]
amount_test = amount_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(amount_train))
print(len(amount_test))


48
12


In [65]:
look_back = 15

train_generator = TimeseriesGenerator(amount_train, amount_train, length=47, batch_size=1)     
test_generator = TimeseriesGenerator(amount_test, amount_test, length=11, batch_size=1)

In [66]:

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25


<ipython-input-66-4381b1b2575e>:14: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



1/1 [==============================] - 2s 2s/step - loss: 71192728.0000
Epoch 2/25
1/1 [==============================] - 0s 44ms/step - loss: 49052788.0000
Epoch 3/25
1/1 [==============================] - 0s 44ms/step - loss: 30269790.0000
Epoch 4/25
1/1 [==============================] - 0s 41ms/step - loss: 21667644.0000
Epoch 5/25
1/1 [==============================] - 0s 43ms/step - loss: 32749376.0000
Epoch 6/25
1/1 [==============================] - 0s 40ms/step - loss: 40665480.0000
Epoch 7/25
1/1 [==============================] - 0s 40ms/step - loss: 40467140.0000
Epoch 8/25
1/1 [==============================] - 0s 54ms/step - loss: 35141508.0000
Epoch 9/25
1/1 [==============================] - 0s 40ms/step - loss: 11987675.0000
Epoch 10/25
1/1 [==============================] - 0s 38ms/step - loss: 10944906.0000
Epoch 11/25
1/1 [==============================] - 0s 40ms/step - loss: 34783408.0000
Epoch 12/25
1/1 [==============================] - 0s 41ms/step - loss: 3030

In [63]:
import plotly.graph_objs as go

In [69]:
prediction = model.predict_generator(test_generator)

amount_train = amount_train.reshape((-1))
amount_test = amount_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = amount_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = amount_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Blood Donation",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "amount"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

<ipython-input-69-77acf6bc2cd5>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [68]:
amount_data = amount_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = amount_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 408ms/step


1/1 [==============================] - 0s 48ms/step
